In [24]:
# install the latest version of the openai library
!pip install openai -q --upgrade

# imports
from openai import OpenAI
import json
import time
import requests


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [25]:
# Load the OpenAI key from a file
with open('../secrets/openAI_key', 'r') as file:
    openAI_key = file.read().replace('\n', '')

# load the github token
with open('../secrets/github_token', 'r') as file:
    github_token = file.read().replace('\n', '')

In [26]:
# initalize the client
client = OpenAI(
    api_key = openAI_key
)

In [40]:
# ASK THE CODE API functions

# Get repository structure
def get_repo_structure(url, branch=None, relativePaths=None):
    # URL of Askthecode API endpoint
    get_repo_structure_url = "https://gabriel.askthecode.ai/api/repository/structure"

    # parameters: url(required), branch(optional), relativePaths(optional)
    params = {
        'url': url,
        'branch': branch,
        'relativePaths': relativePaths
    }

    # headers
    headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {github_token}", 
    "Content-Type": "application/json"
    }

    # Make the post request
    response = requests.post(get_repo_structure_url, json=params, headers=headers)

    # check if the response is successful
    if response.status_code == 200:
        # Parsing the response JSON
        structure_response = response.json()
        # Return the response data instead of printing
        return structure_response
    
    else:
        print(f"Failed to get the repository structure: {response.status_code}")


# get repo content
def get_repo_content(url, filePaths, branch=None, relativePath=None):
    # URL of Askthecode API endpoint
    get_repo_content_url = "https://gabriel.askthecode.ai/api/repository/content"

    # parameters: url(required), filePaths(required), branch(optional), relativePath(optional)
    params = {
        'url': url,
        'filePaths': filePaths,
        'branch': branch,
        'relativePath': relativePath
    }

    # headers
    headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {github_token}", 
    "Content-Type": "application/json"
    }

    # Make the post request
    response = requests.post(get_repo_content_url, json=params, headers=headers)

    # check if the response is successful
    if response.status_code == 200:
        # Parsing the response JSON
        content_response = response.json()
        # Return the response data instead of printing
        return content_response
    
    else:
        print(f"Failed to get the repository content: {response.status_code}")

""""
Test the functions
"""
# Structure retrieval
test_repo_structure_url = get_repo_structure(url="https://github.com/recommenders-team/recommenders")
print("Here is the structure of the repository:")
print(test_repo_structure_url)

# Content retrieval
test_repo_content_url = get_repo_content(url="https://github.com/recommenders-team/recommenders", filePaths=[".github/.codecov.yml"])
print("Here is the content of the repository:")
print(test_repo_content_url)

Here is the structure of the repository:
{'branchName': 'main', 'files': ['.devcontainer/Dockerfile', '.devcontainer/devcontainer.json', '.github/.codecov.yml', '.github/CODEOWNERS', '.github/ISSUE_TEMPLATE.md', '.github/ISSUE_TEMPLATE/bug_report.md', '.github/ISSUE_TEMPLATE/feature_request.md', '.github/ISSUE_TEMPLATE/general-ask.md', '.github/PULL_REQUEST_TEMPLATE.md', '.github/actions/azureml-test/action.yml', '.github/actions/get-test-groups/action.yml', '.github/workflows/azureml-cpu-nightly.yml', '.github/workflows/azureml-gpu-nightly.yml', '.github/workflows/azureml-release-pipeline.yml', '.github/workflows/azureml-spark-nightly.yml', '.github/workflows/azureml-unit-tests.yml', '.github/workflows/sarplus.yml', '.github/workflows/update_documentation.yml', 'AUTHORS.md', 'CODE_OF_CONDUCT.md', 'CONTRIBUTING.md', 'GLOSSARY.md', 'LICENSE', 'MANIFEST.in', 'NEWS.md', 'README.md', 'SECURITY.md', 'SETUP.md', 'contrib/README.md', 'contrib/azureml_designer_modules/README.md', 'contrib/azur

### Assistant API

In [42]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_repo_structure",
            "description": "Retrieves the Github repository file structure to analyze it and be able to query only relevant files. If the provided URL contains specific branch and directory information, prioritize using that over querying the entire repository structure.",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "minLength": 1,
                        "type": "string",
                        "description": "Full Github repository URL provided by the user. For example: https://github.com/[owner]/[repo]/blob/[branch]/[file-path]#[additional-parameters]. The URL MUST be identical to the one, that was provided by the user, you MUST NEVER alter or truncate it. This is crucial for valid responses. You should NEVER truncate additional-parameters.",
                    },
                    "branch": {
                        "type": "string",
                        "description": "Repository branch. Provide only if user has explicitly specified it or the previous plugin response contains it.",
                        "nullable": True
                    },
                    "relativePaths": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "Relative paths to retrieve. USE only paths you are certain that exist, NEVER invent them. If the provided URL contains a specific directory path, extract and use it. Otherwise, this should be a directory path or pattern only. Patterns accept * symbol as 'any substring'",
                        "nullable": True
                    }
            },
            "required": ["url"],
            "additionalProperties": False}
        }
    }

# Add more tools here
]

In [29]:
# Create Assistant
assistant = client.beta.assistants.create(
    name = "codecompass",
    instructions = "You are a helpful assistant that analyzes code from github repositories and files when given a github url. You will answer questions about the structure of a repository, the content of a files, or any other code-related queries.",
    model = "gpt-3.5-turbo-0125",
    tools = tools
)


In [30]:
# Utility function to create a message and run

def create_message_and_run(assistant,query,thread=None):
  if not thread:
    thread = client.beta.threads.create()

  message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=query
  )
  run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
  )
  return run,thread

# Utility function to get details of function to be called

def get_function_details(run):

  print("\nrun.required_action\n",run.required_action)

  function_name = run.required_action.submit_tool_outputs.tool_calls[0].function.name
  arguments = run.required_action.submit_tool_outputs.tool_calls[0].function.arguments
  function_id = run.required_action.submit_tool_outputs.tool_calls[0].id

  print(f"function_name: {function_name} and arguments: {arguments}")

  return function_name, arguments, function_id

# Utility function to submit the function response

def submit_tool_outputs(run,thread,function_id,function_response):
    run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
      {
        "tool_call_id": function_id,
        "output": str(function_response),
      }
    ]
    ) 
    return run

available_functions = {
    "get_repo_structure": get_repo_structure
}

def get_gpt_response(messages):
    chat_completion = client.chat.completions.create(
        model = "gpt-3.5-turbo-0125",
        messages = messages,
        functions = functions,
        function_call = "auto"
    )
    return chat_completion

# execute the function

def execute_function_call(function_name,arguments):
    function = available_functions.get(function_name,None)
    if function:
        arguments = json.loads(arguments)
        results = function(**arguments)
    else:
        results = f"Error: function {function_name} does not exist"
    return results

In [31]:
# test
query = "I want to know about my repository"
run,thread = create_message_and_run(assistant = assistant ,query = query)

In [32]:
run

Run(id='run_GV5GwCtgbmGky0XGlzeT0iht', assistant_id='asst_A2TwKX0j4tonnCBEKQEyxwEW', cancelled_at=None, completed_at=None, created_at=1711472946, expires_at=1711473546, failed_at=None, file_ids=[], instructions='You are a helpful assistant that analyzes code from github repositories and files when given a github url. You will answer questions about the structure of a repository, the content of a files, or any other code-related queries.', last_error=None, metadata={}, model='gpt-3.5-turbo-0125', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_L02tH1oezaPnCkMg4GJDg6mU', tools=[FunctionTool(function=FunctionDefinition(name='get_repo_structure', description='Retrieves the Github repository file structure to analyze it and be able to query only relevant files. If the provided URL contains specific branch and directory information, prioritize using that over querying the entire repository structure.', parameters={'type': 'object', 'properties':

In [33]:
while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id) # retrieve the run status from the thread we created in the previous cell
    print("run status", run.status)

    if run.status=="requires_action":

        function_name, arguments, function_id  = get_function_details(run)

        function_response = execute_function_call(function_name,arguments)

        run = submit_tool_outputs(run,thread,function_id,function_response)

        continue
    if run.status=="completed": # means gpt has an output

        messages = client.beta.threads.messages.list(thread_id=thread.id)
        latest_message = messages.data[0]
        text = latest_message.content[0].text.value
        print(text)

        user_input = input()
        if user_input == "STOP":
          break

        run,thread = create_message_and_run(assistant=assistant,query=user_input,thread=thread)

        continue;
    time.sleep(1)

run status in_progress
run status requires_action

run.required_action
 RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_pdGRlZBbCf2rJfmHK9h1c9mV', function=Function(arguments='{"url":"https://github.com/laravel/laravel"}', name='get_repo_structure'), type='function')]), type='submit_tool_outputs')
function_name: get_repo_structure and arguments: {"url":"https://github.com/laravel/laravel"}
run status in_progress
run status in_progress
run status in_progress
run status completed
I have retrieved the structure of the Laravel repository. It is on the branch `11.x` and contains various files and directories, such as `.editorconfig`, `.env.example`, `app/Http/Controllers/Controller.php`, `app/Models/User.php`, `composer.json`, `database/migrations`, `public`, `resources`, `routes`, `tests`, and more.

If you have any specific files you would like to see the content of, please let me know.
run status in_progress
run statu